# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 3:18PM,249284,10,DNMD-25415,Emerginnova,Released
1,Oct 24 2022 3:18PM,249284,10,Enova-10862,Emerginnova,Released
2,Oct 24 2022 3:18PM,249284,10,Enova-10863,Emerginnova,Released
3,Oct 24 2022 3:18PM,249284,10,Enova-10864,Emerginnova,Released
4,Oct 24 2022 3:07PM,249283,10,8688600,Yusen – 3D Matrix,Released
5,Oct 24 2022 3:02PM,249282,10,0086100461,ISDIN Corporation,Released
6,Oct 24 2022 3:02PM,249282,10,0086100463,ISDIN Corporation,Released
7,Oct 24 2022 3:02PM,249282,10,0086100462,ISDIN Corporation,Released
8,Oct 24 2022 3:02PM,249282,10,0086100464,ISDIN Corporation,Released
9,Oct 24 2022 3:02PM,249282,10,0086100599,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,249280,Executing,1
16,249280,Released,3
17,249282,Released,11
18,249283,Released,1
19,249284,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,NaN,NaN,1.0
249280,NaN,1.0,3.0
249282,NaN,NaN,11.0
249283,NaN,NaN,1.0
249284,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249218,0.0,0.0,1.0
249232,0.0,0.0,1.0
249236,0.0,0.0,2.0
249241,25.0,0.0,2.0
249252,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249218,0,0,1
249232,0,0,1
249236,0,0,2
249241,25,0,2
249252,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249218,0,0,1
1,249232,0,0,1
2,249236,0,0,2
3,249241,25,0,2
4,249252,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249218,,,1
1,249232,,,1
2,249236,,,2
3,249241,25,,2
4,249252,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 3:18PM,249284,10,Emerginnova
4,Oct 24 2022 3:07PM,249283,10,Yusen – 3D Matrix
5,Oct 24 2022 3:02PM,249282,10,ISDIN Corporation
16,Oct 24 2022 2:57PM,249280,19,"Methapharm, Inc."
20,Oct 24 2022 2:55PM,249279,12,Hush Hush
21,Oct 24 2022 2:48PM,249278,19,ACG North America LLC
22,Oct 24 2022 2:40PM,249276,10,"Citieffe, Inc."
25,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation
38,Oct 24 2022 1:46PM,249267,10,ISDIN Corporation
68,Oct 24 2022 1:42PM,249270,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 24 2022 3:18PM,249284,10,Emerginnova,,,4
1,Oct 24 2022 3:07PM,249283,10,Yusen – 3D Matrix,,,1
2,Oct 24 2022 3:02PM,249282,10,ISDIN Corporation,,,11
3,Oct 24 2022 2:57PM,249280,19,"Methapharm, Inc.",,1,3
4,Oct 24 2022 2:55PM,249279,12,Hush Hush,,,1
5,Oct 24 2022 2:48PM,249278,19,ACG North America LLC,,,1
6,Oct 24 2022 2:40PM,249276,10,"Citieffe, Inc.",,,3
7,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,,5,8
8,Oct 24 2022 1:46PM,249267,10,ISDIN Corporation,,,30
9,Oct 24 2022 1:42PM,249270,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 3:18PM,249284,10,Emerginnova,4,,
1,Oct 24 2022 3:07PM,249283,10,Yusen – 3D Matrix,1,,
2,Oct 24 2022 3:02PM,249282,10,ISDIN Corporation,11,,
3,Oct 24 2022 2:57PM,249280,19,"Methapharm, Inc.",3,1,
4,Oct 24 2022 2:55PM,249279,12,Hush Hush,1,,
5,Oct 24 2022 2:48PM,249278,19,ACG North America LLC,1,,
6,Oct 24 2022 2:40PM,249276,10,"Citieffe, Inc.",3,,
7,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,8,5,
8,Oct 24 2022 1:46PM,249267,10,ISDIN Corporation,30,,
9,Oct 24 2022 1:42PM,249270,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 3:18PM,249284,10,Emerginnova,4,,
1,Oct 24 2022 3:07PM,249283,10,Yusen – 3D Matrix,1,,
2,Oct 24 2022 3:02PM,249282,10,ISDIN Corporation,11,,
3,Oct 24 2022 2:57PM,249280,19,"Methapharm, Inc.",3,1,
4,Oct 24 2022 2:55PM,249279,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 3:18PM,249284,10,Emerginnova,4.0,NaN,NaN
1,Oct 24 2022 3:07PM,249283,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Oct 24 2022 3:02PM,249282,10,ISDIN Corporation,11.0,NaN,NaN
3,Oct 24 2022 2:57PM,249280,19,"Methapharm, Inc.",3.0,1.0,NaN
4,Oct 24 2022 2:55PM,249279,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 3:18PM,249284,10,Emerginnova,4.0,0.0,0.0
1,Oct 24 2022 3:07PM,249283,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Oct 24 2022 3:02PM,249282,10,ISDIN Corporation,11.0,0.0,0.0
3,Oct 24 2022 2:57PM,249280,19,"Methapharm, Inc.",3.0,1.0,0.0
4,Oct 24 2022 2:55PM,249279,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2492641,60.0,17.0,0.0
12,249279,1.0,0.0,0.0
15,249241,2.0,0.0,25.0
18,249252,1.0,0.0,0.0
19,997060,6.0,1.0,0.0
20,249265,0.0,21.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2492641,60.0,17.0,0.0
1,12,249279,1.0,0.0,0.0
2,15,249241,2.0,0.0,25.0
3,18,249252,1.0,0.0,0.0
4,19,997060,6.0,1.0,0.0
5,20,249265,0.0,21.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,17.0,0.0
1,12,1.0,0.0,0.0
2,15,2.0,0.0,25.0
3,18,1.0,0.0,0.0
4,19,6.0,1.0,0.0
5,20,0.0,21.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,12,Released,1.0
2,15,Released,2.0
3,18,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20
Status,,,,,,
Completed,0.0,0.0,25.0,0.0,0.0,0.0
Executing,17.0,0.0,0.0,0.0,1.0,21.0
Released,60.0,1.0,2.0,1.0,6.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20
0,Completed,0.0,0.0,25.0,0.0,0.0,0.0
1,Executing,17.0,0.0,0.0,0.0,1.0,21.0
2,Released,60.0,1.0,2.0,1.0,6.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20
0,Completed,0.0,0.0,25.0,0.0,0.0,0.0
1,Executing,17.0,0.0,0.0,0.0,1.0,21.0
2,Released,60.0,1.0,2.0,1.0,6.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()